In [43]:
import folium
import psycopg2
import pandas as pd

In [9]:
dbname = 'routing_db_crime_2'
username = 'nishan'
password = 'vikaspuri'
con = psycopg2.connect(database=dbname, user=username, password=password)

In [10]:
query = """
        SELECT *, ST_AsGeoJSON(the_geom) AS geojson_geometry FROM ways WHERE
	       ST_DWithin(the_geom, ST_GeomFromText('POINT(-73.947494 40.687179)', 4326), 0.05)
        """
df = pd.read_sql(query, con)

In [27]:
df.columns

Index(['gid', 'class_id', 'length', 'length_m', 'name', 'source', 'target',
       'x1', 'y1', 'x2',
       ...
       'cost_crime_offense_class_indirect_bodily_harm_hour_15',
       'cost_crime_offense_class_indirect_bodily_harm_hour_16',
       'cost_crime_offense_class_indirect_bodily_harm_hour_17',
       'cost_crime_offense_class_indirect_bodily_harm_hour_18',
       'cost_crime_offense_class_indirect_bodily_harm_hour_19',
       'cost_crime_offense_class_indirect_bodily_harm_hour_20',
       'cost_crime_offense_class_indirect_bodily_harm_hour_21',
       'cost_crime_offense_class_indirect_bodily_harm_hour_22',
       'cost_crime_offense_class_indirect_bodily_harm_hour_23',
       'geojson_geometry'],
      dtype='object', length=105)

In [44]:
df_tmp = df.loc[:,['gid', 'cost_crime0']]

In [45]:
# normalize between 0 and 1
df_tmp.cost_crime0 = df_tmp.cost_crime0/df_tmp.cost_crime0.max()

In [46]:
# (40.687179, -73.947494)
mymap = folium.Map(location=[40.687179, -73.947494], tiles='https://api.mapbox.com/styles/v1/mapbox/streets-v9/tiles/256/\{z\}/\{x\}/\{y\}?access_token=pk.eyJ1IjoibmFoc2luIiwiYSI6ImNpdDdwdDV0bzA5dHkyeW13ZTh4enl0c3MifQ.iOW2JTxp_HkABm9wuTuPqA', attr='My Data Attribution')

In [47]:
mymap

In [37]:
mymap.choropleth(geo_path='roads.json', data=df_tmp, columns=['gid', 'cost_crime0'], key_on='feature.id', fill_color='YlOrRd', fill_opacity=1.0, line_color='red', line_opacity=1.0, line_weight=2)

/home/nishan/.local/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: 'threshold_scale' default behavior has changed. Now you get a linear scale between the 'min' and the 'max' of your data. To get former behavior, use folium.utilities.split_six.
  if __name__ == '__main__':


In [49]:
df.geojson_geometry[0]

'{"type":"LineString","coordinates":[[-73.9147568,40.6843469],[-73.9174143,40.6840406]]}'

In [53]:
folium.PolyLine([[-73.9147568,40.6843469],[-73.9174143,40.6840406]],color='#0060ff', weight=10, opacity=1.0).add_to(mymap)

In [55]:
folium.PolyLine?

In [ ]:
mymap.save('test.html')